In [1]:
from pyspark.sql import functions

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
23,application_1680595456061_0024,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from py4j.protocol import Py4JJavaError
from pyspark import SparkConf, SparkContext
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import explode, col, udf, concat_ws, from_json, lit, array, expr, size, when, count, avg, sum
# from pyspark.sql.functions import sum as _sum
from pyspark.sql.types import *
from pyspark.sql.types import BooleanType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import json
import os
import gc
import ast


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# %%info


In [5]:
# install vaderSentiment 
sc.install_pypi_package("vaderSentiment")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.list_packages()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- ----------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
certifi                    2022.12.7
charset-normalizer         3.1.0
click                      8.1.3
cycler                     0.11.0
docutils                   0.14
fonttools                  4.38.0
idna                       3.4
jmespath                   1.0.1
joblib                     1.2.0
kiwisolver                 1.4.4
lockfile                   0.11.0
lxml                       4.9.2
matplotlib                 3.5.3
mysqlclient                1.4.2
nltk                       3.8
nose                       1.3.4
numpy                      1.20.0
packaging                  23.0
pandas                     1.3.5
Pillow                     9.5.0
pip                        20.2.2
py-dateutil                2.2
pyparsing                  3.0.9
pystache                   0.5.4
python-daemon              2.2.3
pyt

In [ ]:
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# analyser = SentimentIntensityAnalyzer()

# vader_score = analyser.polarity_scores('I am getting bored in the quarantine')

# print(vader_score)

In [4]:
input_bucket = 's3://cs5344-twitter-project'
original_sdf = spark.read.format('json').options(header='True').load('s3://cs5344-twitter-project/input/original_tweet_intermediate_full').cache()
conn_sdf = spark.read.format('json').options(header='True').load('s3://cs5344-twitter-project/input/retweeted_tweet_intermediate_full').cache()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
pagerank_cand_sdf = spark.read.csv(input_bucket + '/test_pokemon_damping/*.csv').select(col('_c0'))
pagerank_cand_sdf.show()
cand_ls = list(pagerank_cand_sdf.collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|                _c0|
+-------------------+
|          130447415|
|         2786677338|
|          307902310|
| 811765148595535872|
|          323091128|
|           29758446|
| 909119962572242944|
|            5162861|
| 987228577522376704|
|1470439914953011204|
|         3095001124|
|1349579108187992071|
|1441557548427927554|
|1441557548427927554|
|          252087695|
|         1149178590|
|1302006102364745728|
|         2597010860|
|             890891|
|1365337673074933765|
+-------------------+
only showing top 20 rows

In [7]:
print(f'Number of original tweets: {original_sdf.count()}')
print(f'Number of connect tweets: {conn_sdf.count()}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of original tweets: 374556
Number of connect tweets: 860062

In [9]:
original_sdf.filter(col('user_id_str'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+-------------------+----------------------------------+----------------------------------+-------------------+
|_corrupt_record|          created_at|             id_str|                              text|                      text_hashtag|        user_id_str|
+---------------+--------------------+-------------------+----------------------------------+----------------------------------+-------------------+
|           null|Sat Jan 29 12:00:...|1487395140696084480|              ............KK aa...|              KaranKundrra,Kara...|1287296804657090563|
|           null|Sat Jan 29 14:00:...|1487425511630458881|              Eu só quero meu p...|                                  |1163969693209825280|
|           null|Sat Jan 29 14:01:...|1487425645864906753|    異世界といえばスマホ\n #Pok...|              PokemonLEGENDS,Ni...|1034840835961962497|
|           null|Sat Jan 29 15:34:...|1487449041701150722|              no gilgamesh, no ...|                       

In [8]:
@udf
def sentiment_scorer(text):
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()
    score = sia.polarity_scores(text)
    return str(score)


@udf
def get_value_by_key_dict(dict_str, key=None):
    import ast
    dict_str = dict_str.replace('=', ':')
    dict_value = ast.literal_eval(dict_str)
    return dict_value[key]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_sentiment_score(sdf, text_col):
    sdf = sdf.withColumn('sentiment_output', sentiment_scorer(col(text_col)))
    sdf = sdf.withColumn('sentiment_neg', get_value_by_key_dict(col('sentiment_output'), lit('neg')).cast('double'))
    sdf = sdf.withColumn('sentiment_neu', get_value_by_key_dict(col('sentiment_output'), lit('neu')).cast('double'))
    sdf = sdf.withColumn('sentiment_pos', get_value_by_key_dict(col('sentiment_output'), lit('pos')).cast('double'))
    sdf = sdf.withColumn('sentiment_compound', get_value_by_key_dict(col('sentiment_output'), lit('compound')).cast('double'))
    sdf = sdf.withColumn('sentiment_compound_pos_ind', when(sdf.sentiment_compound >= 0, 1).otherwise(0))
    return sdf

In [ ]:
# ###############################################
# ############ Original #################
# ###############################################
original_sdf = get_sentiment_score(original_sdf, 'text')

# # computer average compound score according user's all original tweets
sen_original_sdf = original_sdf.select('user_id_str', 'sentiment_compound', 'sentiment_compound_pos_ind')


sen_original_sdf = sen_original_sdf.groupBy("user_id_str").agg(
    count('*').alias('num_original_tweets'),
    sum('sentiment_compound_pos_ind').alias('num_pos_tweets'),
    avg('sentiment_compound').alias('avg_sentiment_compound')
)


sen_original_sdf.printSchema()
sen_original_sdf.show()



In [ ]:
# ###############################################
# ############ Connect #################
# ###############################################

conn_sdf = get_sentiment_score(conn_sdf, 'text')
conn_sdf.show()

conn_sdf = conn_sdf.groupBy("connected_user_single").agg(
    count('*').alias('num_original_tweets_interacting'),
    sum('sentiment_compound_pos_ind').alias('num_pos_tweets_interacting'),
    avg('sentiment_compound').alias('avg_sentiment_compound_interacting')
)
conn_sdf.printSchema()
conn_sdf.show()

In [ ]:
# spark.stop()